In [24]:
import pandas as pd
from datetime import datetime
import mca
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance

C:\Users\mingt\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Split the file since its too big for github, extract the file outside of .git repository

hpg_res = pd.read_csv('C:\\Users\\mingt\\Downloads\\hpg_reserve.csv\\hpg_reserve.csv')
hpg_res.info()
df1 = hpg_res.iloc[0:100160]
df2 = hpg_res.iloc[100161:200320]
#df1.to_csv('hpg_reserve1.csv')
#df2.to_csv('hpg_reserve2.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
hpg_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors    int64
dtypes: int64(1), object(3)
memory usage: 61.0+ MB


In [3]:
air_reserve = pd.read_csv('air_reserve.csv')
air_store_info= pd.read_csv('air_store_info.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
date_info = pd.read_csv('date_info.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [4]:
#concat files back after its been uploaded on github
hpg_reserve1 = pd.read_csv('hpg_reserve1.csv')
hpg_reserve2 = pd.read_csv('hpg_reserve2.csv')

hpg_reserve = pd.concat([hpg_reserve1, hpg_reserve2])
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])

In [24]:
print(len(store_id_relation))
store_id_relation.head()


150


,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [23]:
print(len(air_reserve))
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [27]:
print(len(air_reserve))
air_reserve.head(5)

92378


TypeError: <class 'pandas.core.series.Series'> is not convertible to datetime

In [18]:
print(len(air_store_info))
air_store_info.head()

829


,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [19]:
print(len(air_visit_data))
air_visit_data.head()

252108


,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [25]:
print(len(hpg_store_info))
hpg_store_info.head()

4690


,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [9]:
print(len(hpg_reserve))
hpg_reserve

200319


,Unnamed: 0,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13
5,5,hpg_28bdf7a336ec6a7b,2016-01-01 17:00:00,2016-01-01 15:00:00,2
6,6,hpg_2a01a042bca04ad9,2016-01-01 17:00:00,2016-01-01 17:00:00,2
7,7,hpg_2a84dd9f4c140b82,2016-01-01 17:00:00,2016-01-01 15:00:00,2
8,8,hpg_2ad179871696901f,2016-01-01 17:00:00,2016-01-01 13:00:00,2
9,9,hpg_2c1d989eedb0ff83,2016-01-01 17:00:00,2016-01-01 15:00:00,6


In [6]:
air_combined = pd.merge(air_reserve, air_store_info, on='air_store_id', how='outer')
hpg_combined = pd.merge(hpg_reserve, hpg_store_info, on='hpg_store_id', how='left')

air_combined.head(5)
hpg_combined.head(5)

,Unnamed: 0,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,hpg_genre_name,hpg_area_name,latitude,longitude
0,0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,NaN,NaN,NaN,NaN
1,1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
2,2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,NaN,NaN,NaN,NaN
3,3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,NaN,NaN,NaN,NaN
4,4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,NaN,NaN,NaN,NaN


In [7]:
print(len(sample_submission))
sub_store = sample_submission['id'].apply(lambda x: str(x).split('_', 2)[:2])
sub_dates = pd.to_datetime(sample_submission['id'].apply(lambda x: str(x).split('_', 2)[2]).rename('Date'))
sub_stores = pd.Series(['_'.join(x) for x in sub_store]).rename('Restaurant')

testing_feats = pd.DataFrame({
        'Restaurant': sub_stores,
        'year': sub_dates.dt.year,
        'month': sub_dates.dt.month,
        'day': sub_dates.dt.day })

testing_feats.head(5)

32019


,Restaurant,day,month,year
0,air_00a91d42b08b08d9,23,4,2017
1,air_00a91d42b08b08d9,24,4,2017
2,air_00a91d42b08b08d9,25,4,2017
3,air_00a91d42b08b08d9,26,4,2017
4,air_00a91d42b08b08d9,27,4,2017


In [8]:
df = store_id_relation.merge(hpg_combined, on='hpg_store_id', how='left')
df2 = air_combined.merge(df, on='air_store_id', how='left')

i = pd.to_datetime(df2['visit_datetime_x'])
df2['all_dates'] = i.dt.date.astype(str)
df2['latitude'] = df2['latitude_x'].astype(str)
df2['longitude'] = df2['longitude_x'].astype(str)
#date_info['calendar_dates'] = pd.to_datetime(date_info['calendar_date'])

date_info['Is_holiday'] = date_info['holiday_flg'].map({1: 'Yes', 0: 'No'})

df3 = pd.merge(df2, date_info, how='left', left_on='all_dates', right_on='calendar_date')

In [8]:
df3.head(2)


,air_store_id,visit_datetime_x,reserve_datetime_x,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id,Unnamed: 0,...,hpg_area_name,latitude_y,longitude_y,all_dates,latitude,longitude,calendar_date,day_of_week,holiday_flg,Is_holiday
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,NaN,2016-01-01,35.6580681,139.7515992,2016-01-01,Friday,1.0,Yes
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,NaN,2016-01-01,35.6580681,139.7515992,2016-01-01,Friday,1.0,Yes


In [20]:
a = pd.to_datetime(df3['visit_datetime_x'])
b = pd.to_datetime(df3['reserve_datetime_x'])
c = pd.to_datetime(df3['visit_datetime_y'])
d = pd.to_datetime(df3['reserve_datetime_y'])



datetime_df =pd.DataFrame({
        'air_visit_year': a.dt.year,
        'air_visit_month': a.dt.month,
        'air_visit_day': a.dt.day,
        'air_visit_hour': a.dt.hour,
        'air_reserve_year': b.dt.year,
        'air_reserve_month': b.dt.month,
        'air_reserve_day': b.dt.day,
        'air_reserve_hour': b.dt.hour,
         
        'hpg_visit_year': c.dt.year,
        'hpg_visit_month': c.dt.month,
        'hpg_visit_day': c.dt.day,
        'hpg_visit_hour': c.dt.hour,
        'hpg_reserve_year': d.dt.year,
        'hpg_reserve_month': d.dt.month,
        'hpg_reserve_day': d.dt.day,
        'hpg_reserve_hour': d.dt.hour        
    })



In [29]:
categorical_df = df3[['air_store_id', 'air_genre_name', 'air_area_name', 
                    'hpg_genre_name', 'hpg_area_name', 'day_of_week', 'Is_holiday']].fillna('None_Stated')
#'latitude', 'longitude',


dummy_df = pd.get_dummies(categorical_df)
combined_df = pd.concat([datetime_df, dummy_df], axis=1)

air_visitors = df3['reserve_visitors_x'].fillna(0)
hpg_visitors = df3['reserve_visitors_y'].fillna(0)
df3['reserve_total'] = air_visitors + hpg_visitors

target = df3['reserve_total']

In [ ]:
xgb.train

In [25]:
#http://okomestudio.net/biboroku/?p=2357
class CA(object):
    """Simple corresondence analysis.
    
    Inputs
    ------
    ct : array_like
      Two-way contingency table. If `ct` is a pandas DataFrame object,
      the index and column values are used for plotting.
    Notes
    -----
    The implementation follows that presented in 'Correspondence
    Analysis in R, with Two- and Tghree-dimensional Graphics: The ca
    Package,' Journal of Statistical Software, May 2007, Volume 20,
    Issue 3.
    """

    def __init__(self, ct):
        self.rows = ct.index.values if hasattr(ct, 'index') else None
        self.cols = ct.columns.values if hasattr(ct, 'columns') else None
        
        # contingency table
        N = np.matrix(ct, dtype=float)

        # correspondence matrix from contingency table
        P = N / N.sum()

        # row and column marginal totals of P as vectors
        r = P.sum(axis=1)
        c = P.sum(axis=0).T

        # diagonal matrices of row/column sums
        D_r_rsq = np.diag(1. / np.sqrt(r.A1))
        D_c_rsq = np.diag(1. / np.sqrt(c.A1))

        # the matrix of standarized residuals
        S = D_r_rsq * (P - r * c.T) * D_c_rsq

        # compute the SVD
        U, D_a, V = svd(S, full_matrices=False)
        D_a = np.asmatrix(np.diag(D_a))
        V = V.T

        # principal coordinates of rows
        F = D_r_rsq * U * D_a

        # principal coordinates of columns
        G = D_c_rsq * V * D_a

        # standard coordinates of rows
        X = D_r_rsq * U

        # standard coordinates of columns
        Y = D_c_rsq * V

        # the total variance of the data matrix
        inertia = sum([(P[i,j] - r[i,0] * c[j,0])**2 / (r[i,0] * c[j,0])
                       for i in range(N.shape[0])
                       for j in range(N.shape[1])])

        self.F = F.A
        self.G = G.A
        self.X = X.A
        self.Y = Y.A
        self.inertia = inertia
        self.eigenvals = np.diag(D_a)**2

MemoryError: 